# Descripción:

Análisis de datos, ingeniería de variables, y modelamiento a través de Pycaret

In [ ]:
import os
os.chdir('C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto')

In [ ]:
u=!pip freeze
instalados=str(u)

In [ ]:
to_instalar=['os','pandas','matplotlib','statsmodels','numpy','seaborn','pingouin','pycaret','sklearn','scipy','openpyxl','optuna',
            'xgboost','sns']

In [ ]:
instalados=[x for x in to_instalar if x in  instalados ]
if len(set(to_instalar)-set(instalados))>0:
    !pip install -r requirements_pycaret.txt 

In [ ]:
import pycaret

In [ ]:
import optuna

In [ ]:
#!pip install -r requirements.txt --user
import os
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from datetime import datetime
import math
import unicodedata
from unicodedata import normalize
import re
from dateutil.relativedelta import relativedelta
import pingouin
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import scipy
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
#import tune_sklearn
#import pycaret

In [ ]:
ruta_madre="C:/Users/USUARIO/Documents/GitHub/Team_224/Proyecto"
ruta_insumos=os.path.join(ruta_madre, 'Insumos')
ruta_resultados=os.path.join(ruta_madre, 'Resultados')

## 1.0 Importando base de datos preprocesada

Tomamos la data proveniente del EDA y la usamos para nuestro ejercicio de modelaje

In [ ]:
os.chdir(os.path.join(ruta_insumos,'preproccess_data'))
ruta_data_preproc=os.path.join(os.getcwd(),'data_agregada_preproccess.xlsx')
datos_preproc=pd.read_excel(ruta_data_preproc,index_col=0)


## 2.0 Featuring engineering 

Se empiza a hacer Feature engineering para evitar resultados raros en el modelaje y mejorar el desempeño del mismo

### 2.1 Deflactando los salarios

Debido al gran problema que genera la inflación a lo largo del tiempo, se prefiere usar cifras deflactadas, con lo cual, se utiliza la tabal abstraida del DANE para el IPC anual y se indexa hasta valores correspondientes al 2022. Es decir, la serie se deflacta y se pone todo en las mismas unidades monetarias en el mismo intervalo de tiempo

In [ ]:
inflacion_variacion=pd.read_excel(os.path.join(os.getcwd(),'inflacion.xlsx'),index_col=0)
inflacion_variacion['inflacion']=inflacion_variacion['inflacion']/100+1
inflacion_variacion=inflacion_variacion[inflacion_variacion.index>=datos_preproc.year.min()]

In [ ]:
inflacion_variacion=inflacion_variacion.sort_index(ascending=False).cumprod().reset_index()

In [ ]:
datos_preproc=datos_preproc.merge(inflacion_variacion,on="year",how="left")
datos_preproc.inflacion.fillna(1,inplace=True)

In [ ]:
datos_preproc['wage_deflacted']=datos_preproc['wage_imputed']*datos_preproc['inflacion']

### 2.2 Arreglando variables categoricas incompletas

Algunas variables sufren de datos faltantes dificiles de imputar y de High Cardinality que puede afectar el balanceo de las variables explicativas. Para ello, se agrupan algunas categorias con el fin de solventar estos datos atípicos

In [ ]:
datos_preproc['total_depedants']=datos_preproc['other_on_charge_person']+datos_preproc['children_amount']

In [ ]:
datos_to_model=datos_preproc[['month','gender','is_special_population','any_disability','MUNICIPIO_DE_RESIDENCIA','home_type',
                             'education_level','PROFESION','marital_status','total_depedants','ESTRATO_SOCIAL','wage_deflacted',
                             'age','years_exp_current_role','request_attend_per_day']].copy()
datos_to_model['total_depedants']=datos_to_model['total_depedants'].astype('str')
datos_to_model['ESTRATO_SOCIAL']=datos_to_model['ESTRATO_SOCIAL'].astype('str')

In [ ]:
datos_to_model.isna().sum().to_frame()

Note que hay un total importante de `NAs` en algunos datos. sin embargo, no tenemos ninguna explicación de negocio para reemplazarlos por un numero, por lo que se pondrá como una nueva categoria

In [ ]:
datos_to_model['gender']=datos_to_model['gender'].fillna('OTHER')
datos_to_model['is_special_population']=datos_to_model['is_special_population'].fillna('NO APLICA')
datos_to_model['any_disability']=datos_to_model['any_disability'].fillna('no_reported')
datos_to_model['MUNICIPIO_DE_RESIDENCIA']=datos_to_model['MUNICIPIO_DE_RESIDENCIA'].fillna('NO_REPORTADO')
datos_to_model['home_type']=datos_to_model['home_type'].fillna('OTHER')
datos_to_model['education_level']=datos_to_model['education_level'].fillna('OTHER')
datos_to_model['PROFESION']=datos_to_model['PROFESION'].fillna('OTHER')
datos_to_model['marital_status']=datos_to_model['marital_status'].fillna('OTHER')
datos_to_model['total_depedants']=datos_to_model['total_depedants'].fillna('OTHER')
datos_to_model['ESTRATO_SOCIAL']=datos_to_model['ESTRATO_SOCIAL'].fillna('OTHER')

El siguiente código presenta una transformacion de la variable edad, ya que al verlo respecto a la variable que mide la productividad, tiene una relacion no lineal. 

### 2.3 Agregando polinomios de la edad

Como se vió en el EDA, la edad tiene ua relación no lineal ni cuadrática con respecto a la variable `request_attend_per_day`

In [ ]:
datos_to_model['age2']=datos_to_model['age']**2
datos_to_model['age3']=datos_to_model['age']**3
datos_to_model['age4']=datos_to_model['age']**4
datos_to_model['age5']=datos_to_model['age']**5

## 3.0 Modelo de referencia de Regresión lineal (baseline model)

### 3.1 Dividiendo en test y train set

Utilzamos un split de 70/30, sin embargo, solo usaremos este ejercicio con fin de tener una referencia de potenciales variables significativas. siendo el peor modelo y luego ir viendo como se comporta a futuro con modelos más sofisticados

In [ ]:
y=datos_to_model[datos_to_model.columns[datos_to_model.columns=='request_attend_per_day']].copy()
X=datos_to_model[datos_to_model.columns[datos_to_model.columns!='request_attend_per_day']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

  ### 3.2 Normalizando y transformando las variables

Normalizaremos salario y años de experiencia y todas las variables de edad mediante una estandarización

In [ ]:
scaler = StandardScaler()

stand_xtrain=scaler.fit_transform(X_train[['wage_deflacted','years_exp_current_role','age','age2','age3','age4','age5']])
X_train[['wage_deflacted','years_exp_current_role','age','age2','age3','age4','age5']]=pd.DataFrame(stand_xtrain,columns=['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5'],
                                                                                                 index=X_train.index)

stand_xtest=scaler.fit_transform(X_test[['wage_deflacted','years_exp_current_role','age','age2','age3','age4','age5']])
X_test[['wage_deflacted','years_exp_current_role','age','age2','age3','age4','age5']]=pd.DataFrame(stand_xtest,columns=['wage_imputed','years_exp_current_role','age','age2','age3','age4','age5'],
                                                                                                index=X_test.index)


### 3.3 Aplicando un box cox

Se aplica la transformación Box-Cox sobre la variable `request_attend_per_day` una vez ya se crea el train set y el test set

In [ ]:
from scipy import stats
ytrain_transformed, box_cox_lambda_ytrain = stats.boxcox(y_train.request_attend_per_day)
ytest_transformed, box_cox_lambda_ytest = stats.boxcox(y_test.request_attend_per_day)

In [ ]:
sns.set_theme()
sns.set_style('white')
plt.figure(figsize=(10,15))

plt.subplot(2,2,1)
sns.histplot(ytrain_transformed)
plt.title('Response variable train Box-cox transformed ')

plt.subplot(2,2,2)
sns.histplot(y_train.request_attend_per_day)
plt.title('Response variable train Original')

plt.subplot(2,2,3)
sns.histplot(ytest_transformed)
plt.title('Response variable test Box-cox transformed')

plt.subplot(2,2,4)
sns.histplot(y_test.request_attend_per_day)
plt.title('Response variable test Original')

La gráfica anterior muestra que sí es útil hacer la transformacion de **Box-Cox**, ya que le disminuye el sesgo y mejorar la distribución de lso datos. por lo cual, deberíamos aplicarla.

In [ ]:
#X_train_extended = sm.add_constant(X_train)
X_train_extended = X_train.copy()
X_train_extended['month']=X_train_extended.month.astype('category')
X_train_extended['gender']=X_train_extended.gender.astype('category')
X_train_extended['is_special_population']=X_train_extended.is_special_population.astype('category')
X_train_extended['any_disability']=X_train_extended.any_disability.astype('category')
X_train_extended['MUNICIPIO_DE_RESIDENCIA']=X_train_extended.MUNICIPIO_DE_RESIDENCIA.astype('category')
X_train_extended['home_type']=X_train_extended.home_type.astype('category')
X_train_extended['PROFESION']=X_train_extended.PROFESION.astype('category')
X_train_extended['marital_status']=X_train_extended.marital_status.astype('category')
X_train_extended['total_depedants']=X_train_extended.total_depedants.astype('category')
X_train_extended['ESTRATO_SOCIAL']=X_train_extended.ESTRATO_SOCIAL.astype('category')
X_train_extended['education_level']=X_train_extended.education_level.astype('category')

In [ ]:
X_train_extended.info()

In [ ]:
data_train=X_train_extended.copy()
data_train['request_per_day']=ytrain_transformed

In [ ]:
data_train.info()

In [ ]:
## usar esto para crear la fórmula dentro del modelo
formula='request_per_day~'
for i in range(len(data_train.columns)):
    if i ==0 and data_train.columns[i]!='request_per_day':
        formula= formula + data_train.columns[i]
    elif data_train.columns[i]!='request_per_day' and i !=0:
        formula= formula+'+'+ data_train.columns[i]
    else:
        continue
    
    

In [ ]:
linear_model_baseline=smf.ols(formula=formula,data=data_train).fit() 

In [ ]:
linear_model_baseline.summary()

In [ ]:
linear_model_baseline.pvalues.index[np.where(linear_model_baseline.pvalues<0.05/len(linear_model_baseline.pvalues))].to_frame().reset_index(drop=True)

In [ ]:
linear_model_baseline.fvalue ### valor del ANOVA agregado del modelo.

In [ ]:
anova_test=sm.stats.anova_lm(linear_model_baseline,typ=1)

In [ ]:
print(anova_test)

In [ ]:
#model_region_no_oldest_box_cox=smf.ols(
 #   formula='Cost_BC_transformed~Region+group_size+homeowner+car_age+car_value+age_youngest+married_couple+C_previous+duration_previous+A+E+F+G',
  #  data=train_expanded).fit()

En la regresión ejecutada arriba, se puede observar que el R squared se acerca al 58% de explicación, el cual no  no es malo, pero definitivamente no es el mejor; del mismo modo, haciendo pruebas de hipotesis individuales aplicando la corección de **Bonferroni**, se observa que las únicas variables que individualmente se eliminan por baja significancia estadística es la variable categoria `Months` y el `wage_deflacted`. Este último puede estar atado al hecho de que el cargo es estandar y de baja cualificación profesional, haciendo que que la diferencia de salarios a lo largo de los cargos no sea significativa. De igual manera, note que al tener una variable dependiente continua y casi normal (es distribución), es posible ver el análsis ANOVA, la cual muestra que en general, alguna o varias de los betas son diferentes de cero.


Dado lo anterior, se puede observar que con las variables significativas se puede volver a correr un modelo de la siguiente forma:

In [ ]:
formula2='request_per_day~gender+is_special_population+any_disability+MUNICIPIO_DE_RESIDENCIA+home_type+education_level+PROFESION+marital_status+total_depedants+ESTRATO_SOCIAL+age+years_exp_current_role+age2+age3+age4+age5'

In [ ]:
data_train.education_level.unique()

In [ ]:
linear_model_baseline_mod=smf.ols(formula=formula2,data=data_train).fit() 
linear_model_baseline_mod.summary()

Con base en el modelo base, podemos encontrar un perfil potencial base que nos dice que: la mayor productividad puede ser de un perfil que sea hombre, que no tenga ninguna condición especial (o provenga del conflicto armado) que provenga de Cereté (Cordoba), que viva en casa familiar, con un nivel profesional por fuera de los tradicionales y que tenga profesiones relacionadas a secretariado y/o asistete administrativo, donde como maximo tenga 3 dependientes (probablemente 8 dependientes observados en el model muestra una significacia grande por temas a pocos datos), con estrato social 3 y con mayor años de experiencia. La edad es un factor positivamente fundamental para la productividad, pero va disminuyendo marginalmente a medida que aumenta.

Debido a que este modelo es de referencia, decidimos no hacer ninguna prueba de supuestos por el momento y ver primero si logra desempeñarse bien frente a otros modelos de ML, más flexible en sus supuestos

## 4.0 Encontrando el modelo adecuado

Se empiza a hacer las investigaciones avanzadas para determinar cual es el mejor modelo

### 4.1 Modelo de regresión

Iniciamos con modelos de regresión utilizando la herramienta built-in en python llamada de pycaret

In [ ]:
datos_to_model.info()

### 4.1.1 Setting up the parameters

Notará que la siguiente linea permite que pycaret haga gran parte pornosotros, primero le relacionamos las variables que serán categóricas y permitimos que:
1) se estandarice (mediante z score) las variables numéricas
2) ignorará las variables que posean 0 varianza aproximadamente
3) permitirá crear una gama e variables más grande, a través de combinaciones de todas las variables. 
4) elimina una de las variables, de aquel par que poseauna correlación de 85% o más
5) como se vio anteriormente, la transformación de la variable respuesta puede ser útil, por lo que permitimos hacer la transformación de Box-cox
6) Se implementa una combinación de Random_forest y regresiones lineales para escoger solo las variables más representativas.

In [ ]:
datos_to_model=datos_to_model.drop(columns='month')


In [ ]:
datos_to_model.to_excel(os.path.join(ruta_insumos,'master_data','data_master_ready_to_model.xlsx'))

In [ ]:
from pycaret.regression import *
set_model = setup(datos_to_model, target = 'request_attend_per_day',categorical_features=['gender','is_special_population','any_disability','MUNICIPIO_DE_RESIDENCIA','home_type','education_level',
                                                                                  'marital_status','PROFESION','total_depedants','ESTRATO_SOCIAL'],
          normalize=True,normalize_method='zscore',feature_interaction=True,ignore_low_variance=True,transform_target=True,transform_target_method='box-cox',
          feature_selection=True,feature_selection_threshold=0.85,remove_multicollinearity=True,multicollinearity_threshold=0.85,session_id=1229,train_size=0.70,fold_strategy='kfold',
                 fold=10)

Para este caso, corremos el modelo y seleccionamos el mejor de acuerdo al que tenga el mínimo RMSE. Para ello, se observa que el Extra Tree para regresión es el menor, mejorando en promedio en casi 4 unidades el modelo de regresión lineal y casi aumentando en 20 puntos porcnentales su R2.

In [ ]:
best=compare_models(sort='RMSE')

In [ ]:
print(best)

In [ ]:
modelo_seleccionado=create_model(best,criterion='mse') ## se crea el modelo minimizando el MSE sobre el modelo con menor RMSE

In [ ]:
RMSE_best_model_notuned=pull()

In [ ]:
RMSE_notuned=RMSE_best_model_notuned.loc['Mean'].RMSE

A pesar que Extra Tree parece ser el más útil para este caso, tendríamos un problema y sería al establecer todos los hiperparámetros que requiere este método. Para este caso, se utiliza el proceso de 'tuning' que trata de optimizar el RMSE (minimizarlo) de acuerdo al conjunto de hiperparámetros que serán escogidos a través de optimización bayesiana.

### 4.1.2 Tuneando el modelo


Acá se planea optimizar los hiperparámetros del modelo elegido para ver si es posible mejorar su rendimiento


In [ ]:
#pip install tune-sklearn ray[tune]* 
#modelo_tuneado=tune_model(modelo_seleccionado, search_library = 'tune-sklearn', search_algorithm = 'hyperopt',choose_better=True,optimize='RMSE',n_iter=30)

modelo_tuneado=tune_model(modelo_seleccionado, search_library = 'optuna',choose_better=True,optimize='RMSE',n_iter=30)

In [ ]:
modelo_seleccionado_tuneado=create_model(modelo_tuneado,criterion='mse')

In [ ]:
model_tuned_results = pull()

In [ ]:
model_selected_tuned=model_tuned_results.loc['Mean'].RMSE

Se saca el modelo tuneado y se compara con el sin tunear, logrando así escoger el que más bajo RMSE tenga

In [ ]:
if model_selected_tuned>RMSE_notuned:
    final_model=model_selected_tuned
else:
    final_model=modelo_seleccionado

La siguiente permite crear ficheros y configuraciones para crear las imágenes que se van a correr en el Front-End para evitar gastos innecesarios computacionalmente hablando

In [ ]:
os.chdir('C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Codigo\\Modelling')
all_data_final_model=finalize_model(final_model)
save_model(all_data_final_model, 'C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Models\\model_selected_pipeline')


In [ ]:
save_config('C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Models\\model_selected_config')


In [ ]:
import joblib
joblib.dump(final_model,'C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Models\\final_model_visualization.data')

### 5 Predicción del modelo

Una vez se escoge el modelo, lo que se sugiere hacer es utilizar el modelo ganador sobre toda la data, y en el Front-End, se relacione las variables `gender`,`is_special_population`,`any_disability`,`MUNICIPIO_DE_RESIDENCIA`,`home_type`, `education_level`,`PROFESION`,`marital_status`,`total_depedants`,`ESTRATO_SOCIAL`, `age`, `years_exp_current_role`. Donde el sueldo será puesto internamente por la entidad de salud y las varables polinómicas de la edad, serán calculadaas ua vez se relacione la edad del candidato potencial.	 Para las variables `age`, `years_exp_current_role` su tipo de dato son numéricas y `ESTRATO_SOCIAL`, `total_depedants` parecen numericas pero tambien son categoricas ordinales.

**Pronostico de prueba**
Para este caso. se hará un pronóstico de prueba con el fin de probar el modelo escogido.

In [ ]:
#os.chdir('C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Codigo\\Modelling')
all_data_final_model=load_model('C:\\Users\\USUARIO\\Documents\\GitHub\\Team_224\\Proyecto\\Models\\model_selected_pipeline')


In [ ]:
##load_model('model_selected_pipeline')## para cargar de vuelta el modelo
age=50
gender='FEMALE'
is_special_population='NO APLICA'
municipio_living='monteria'
home_type='ARRENDADA'
education_level='PROFESIONAL'
PROFESION='admin_empresas'
marital_status='SINGLE'
total_dependants='1.0'
ESTRATO_SOCIAL='1.0'
years_exp_current_role=10

In [ ]:
new_data=datos_to_model.iloc[[0,]].copy()
new_data=new_data.drop(columns='request_attend_per_day')
new_data['age']=float(age)
new_data['age2']=float(age)**2
new_data['age3']=float(age)**3
new_data['age4']=float(age)**4
new_data['age5']=float(age)**5
new_data['is_special_population']=is_special_population
new_data['municipio_living']=municipio_living
new_data['home_type']=home_type
new_data['education_level']=education_level
new_data['marital_status']=marital_status
new_data['total_dependants']=str(total_dependants)
new_data['ESTRATO_SOCIAL']=str(ESTRATO_SOCIAL)
new_data['years_exp_current_role']=float(years_exp_current_role)
new_data['PROFESION']=PROFESION

In [ ]:
predicted_value=predict_model(all_data_final_model, data = new_data)

In [ ]:
print(float(predicted_value.Label)) ### este es los requerimientos atendidos por dia que tendrá esa persona

In [ ]:
unicos={}
for i in datos_to_model.select_dtypes('object' or 'category').columns.to_list():
    unicos[i]=datos_to_model[i].unique()


Se crea un fichero para guardar las categorias que se usarán en el Front-End

In [ ]:
import joblib
joblib.dump(unicos,'unicos_categorias.data')

### 5.1 Determinar si un potencial candidato se debería contratar

La idea es transformar la variable respuesta de toda la data mediante el método de **Box-Cox**, para luego tomar el cuantil 75, 50 y 25 y ese valor volver a transformarlo para saber los puntos de corte que la entidad de salud considera para saber si un empleado es o no productivo

In [ ]:
y_transformada, box_cox_lambda_y = stats.boxcox(datos_to_model.request_attend_per_day)

In [ ]:
prob_75=np.quantile(y_transformada,0.75)
prob_50=np.quantile(y_transformada,0.5)
prob_90=np.quantile(y_transformada,0.90)

In [ ]:
valores_quantiles=scipy.special.inv_boxcox(np.asarray([prob_50,prob_75,prob_90]), box_cox_lambda_y)

In [ ]:
valores_quantiles

In [ ]:
#if predicted_value.Label>valores_quantiles[2] :
#    print('Excelente perfil. contrátalo')
#elif (predicted_value.Label>valores_quantiles[1] and predicted_value.Label<=valores_quantiles[2]):
#    print('Es un perfil sobresaliente. contrátalo')
#elif (predicted_value.Label>valores_quantiles[0] and predicted_value.Label<=valores_quantiles[1]):
#    print('Es un perfil estandar. podrías con periodo de prueba')
#else:
#    print('No es tan productivo. ten cuidado')

In [ ]:
import session_info

In [ ]:
session_info.show(html=False)